<a href="https://colab.research.google.com/github/filipeverrone/utilities/blob/main/GeometricComplexNumbersFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def is_number(value): return type(value) in [type(0), type(1.)]

In [47]:
class Matrix:

    def __init__(self, rows: list):
        lines = []
        if len(rows):
            lines = rows

        self.matrix = lines
        self.nlines = len(lines)
        self.ncolumns = len(lines[0])

    def verify_square(self):
        if self.nlines != self.ncolumns:
            raise Exception('this matrix is not a square')

    def transpose(self, save=False):
        result = [[None] * self.nlines for column in range(self.ncolumns)]
        for line in range(self.nlines):
            for column in range(self.ncolumns):
                result[column][line] = self.matrix[line][column]
        if save: self.matrix = result
        return Matrix(result)

    def const_multiply(self, const):
        result = [[None] * self.ncolumns for line in range(self.nlines)]
        for line in range(self.nlines):
            for column in range(self.ncolumns):
                result[line][column] = self.matrix[line][column] * const
        return Matrix(result)

    def __rmul__(self, m2): return self * m2

    def __mul__(self, m2):
        if is_number(m2): return self.const_multiply(m2)

        if self.ncolumns != m2.nlines:
            raise Exception('m2 is not compatible with this matrix!')
        result = []
        for line in range(self.nlines):
            row = []
            for result_line in range(self.nlines):
                item = 0
                for column in range(self.ncolumns):
                    item += self.matrix[line][column] * \
                            m2.matrix[column][result_line]
                row.append(item)
            result.append(row)
        return Matrix(result)

    def __pow__(self, n):
        self.verify_square()

        if n == -1: return self.inverse()        

        response = self
        for i in range(n - 1):
            response *= self

        return response

    def __str__(self):
        response = ''

        for line in range(self.nlines):
            for column in range(self.ncolumns):
                  if column == 0: response += '[ '
                  is_last = column + 1 == self.ncolumns
                  last_chars = '\t' if not is_last else ' ]\n'
                  response += f'{self.matrix[line][column]}{last_chars}'

        return response

    def __add__(self, m2):
        if self.nlines != m2.nlines or self.ncolumns != m2.ncolumns:
            raise Exception('m2 dimensions is not compatible with this matrix!')
        new_matrix = [[None] * self.ncolumns for line in range(self.nlines)]
        for line in range(self.nlines):
            for column in range(self.ncolumns):
                new_matrix[line][column] = self.matrix[line][column] + \
                                            m2.matrix[line][column]
        return Matrix(new_matrix)

    def __radd__(self, m2): return self + m2

    def __sub__(self, m2): return self + (-1) * m2

    def __rsub__(self, m2): return (-1) * self + m2

    def inverse(self):
        determinant = self.determinant()
        if determinant == 0: raise Exception('the determinant is zero!')
        if type(determinant) != type(0): raise Exception('det is not a number!')
        return (1 / determinant) * self.adjunct().transpose()

    def determinant(self):
        self.verify_square()
        response = 0

        if self.nlines == 2:
            main_product = self.get(0, 0) * self.get(1, 1)
            secondary_product = self.get(1, 0) * self.get(0, 1)
            return main_product - secondary_product

        for line in range(self.nlines):
            product = 1
            for column in range(self.ncolumns):
                product *= self.get(line + column, column)
            response += product

        for line in range(self.nlines - 1, -1, -1):
            product = 1
            for column in range(self.ncolumns - 1, -1, -1):
                product *= self.get(line - column, column)
            response -= product

        return response

    def get(self, line, column):
        x, y = line, column
        if line + 1 > self.nlines: x = line % self.nlines
        if column + 1 > self.ncolumns: y = column % self.ncolumns
        return self.matrix[x][y]

    def matrix_except(self, line = 0, column = 0):
        result = []
        for mline in range(self.nlines):
            if mline != line:
                line_result = []
                for mcolumn in range(self.ncolumns):
                    if mcolumn != column:
                        line_result.append(self.get(mline, mcolumn))
                result.append(line_result)
        return Matrix(result)

    def adjunct(self):
        self.verify_square()
        if self.nlines == 2:
            return Matrix([
                [self.matrix[1][1], - self.matrix[1][0]],
                [- self.matrix[0][1], self.matrix[0][0]],
            ])
        adjunct = [[None] * self.ncolumns for line in range(self.nlines)]
        for line in range(self.nlines):
            for column in range(self.ncolumns):
                item_coefficient = (-1) ** (line + column)
                pivot_matrix = self.matrix_except(line, column)
                item = item_coefficient * pivot_matrix.determinant()
                adjunct[line][column] = item
        return Matrix(adjunct)


In [48]:
a = Matrix([
    [1, 1],
    [0, 1]
])

In [49]:
b = Matrix([
    [1, 0],
    [0, 1],
    [1, 0]
])

In [50]:
print((b * b.transpose()))

[ 1	0	1 ]
[ 0	1	0 ]
[ 1	0	1 ]



In [51]:
print(a * 2 + a)

[ 3	3 ]
[ 0	3 ]



In [52]:
a.determinant()

1

In [53]:
(Matrix([
    [1, 1, 1],
    [0, 0, 1],
    [1, 0, 1],
]) + (b * b.transpose())).determinant()

2

In [54]:
Matrix([
    [1, 1, 1],
    [0, 0, 1],
    [1, 0, 1],
]).determinant()

1

In [55]:
print(a.inverse())

[ 1.0	-1.0 ]
[ 0.0	1.0 ]



In [56]:
print(a.inverse() * a)

[ 1.0	0.0 ]
[ 0.0	1.0 ]



In [57]:
c = Matrix([
    [1, 2, 0],
    [2, 2, 1],
    [1, 1, 1],
])

In [58]:
print(c)

[ 1	2	0 ]
[ 2	2	1 ]
[ 1	1	1 ]



In [59]:
c.determinant()

-1

In [60]:
print(c.inverse())

[ -1.0	2.0	-2.0 ]
[ 1.0	-1.0	1.0 ]
[ -0.0	-1.0	2.0 ]



In [61]:
print(c * c.inverse())

[ 1.0	0.0	0.0 ]
[ 0.0	1.0	0.0 ]
[ 0.0	0.0	1.0 ]

